Desenvolvimento do modelo de predicao

In [3]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor

In [5]:
ds = datasources.get('fabioebner/fiap-mlops', 'processed')

In [7]:
ds.all().dataframe

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,laptop-price-brl-processed.csv,100659799,https://dagshub.com/api/v1/repos/fabioebner/fi...,text/plain,48638


In [ ]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url
    print(f"Dataset URL: {dataset_url}")

Output()

Dataset URL: https://dagshub.com/api/v1/repos/fabioebner/fiap-mlops/raw/main/data/processed/laptop-price-brl-processed.csv


In [9]:
df = pd.read_csv(dataset_url)

In [10]:
df.head()

,ram_gb,ssd,hdd,graphic_card_gb,warranty,price,brand_apple,brand_asus,brand_avita,brand_msi,...,os_windows,weight_casual,weight_gaming,weight_thinnlight,touchscreen_0,touchscreen_1,ram_type_ddr4,ram_type_other,os_bit_32-bit,os_bit_64-bit
0,4,0,1024,0,0,2321,0,1,0,0,...,1,1,0,0,1,0,1,0,0,1
1,4,0,1024,0,0,2613,0,0,0,0,...,1,1,0,0,1,0,1,0,0,1
2,4,0,1024,0,0,2680,0,0,0,0,...,1,1,0,0,1,0,1,0,0,1
3,8,512,0,2,0,4689,0,1,0,0,...,1,1,0,0,1,0,1,0,1,0
4,4,0,512,0,0,1808,0,1,0,0,...,1,1,0,0,1,0,1,0,0,1


## Desenvolvimento dos modelos

In [12]:
dagshub.init(repo_owner="fabioebner", repo_name="fiap-mlops", mlflow=True)

Initialized MLflow to track repo "fabioebner/fiap-mlops"

Repository fabioebner/fiap-mlops initialized!

In [13]:
mlflow.autolog()

2025/06/11 20:23:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/06/11 20:23:51 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/06/11 20:23:51 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [14]:
features = list(df.columns)

features.remove("price")

features

['ram_gb',
 'ssd',
 'hdd',
 'graphic_card_gb',
 'warranty',
 'brand_apple',
 'brand_asus',
 'brand_avita',
 'brand_msi',
 'brand_other',
 'processor_brand_amd',
 'processor_brand_intel',
 'processor_brand_m1',
 'processor_name_core i3',
 'processor_name_core i5',
 'processor_name_other',
 'os_other',
 'os_windows',
 'weight_casual',
 'weight_gaming',
 'weight_thinnlight',
 'touchscreen_0',
 'touchscreen_1',
 'ram_type_ddr4',
 'ram_type_other',
 'os_bit_32-bit',
 'os_bit_64-bit']

In [15]:
X = df[features]

In [16]:
len(features)

27

In [17]:
X

,ram_gb,ssd,hdd,graphic_card_gb,warranty,brand_apple,brand_asus,brand_avita,brand_msi,brand_other,...,os_windows,weight_casual,weight_gaming,weight_thinnlight,touchscreen_0,touchscreen_1,ram_type_ddr4,ram_type_other,os_bit_32-bit,os_bit_64-bit
0,4,0,1024,0,0,0,1,0,0,0,...,1,1,0,0,1,0,1,0,0,1
1,4,0,1024,0,0,0,0,0,0,1,...,1,1,0,0,1,0,1,0,0,1
2,4,0,1024,0,0,0,0,0,0,1,...,1,1,0,0,1,0,1,0,0,1
3,8,512,0,2,0,0,1,0,0,0,...,1,1,0,0,1,0,1,0,1,0
4,4,0,512,0,0,0,1,0,0,0,...,1,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,4,1024,0,0,1,0,1,0,0,0,...,1,1,0,0,1,0,1,0,0,1
764,4,1024,0,0,1,0,1,0,0,0,...,1,1,0,0,1,0,1,0,0,1
765,4,1024,0,4,1,0,1,0,0,0,...,1,1,0,0,1,0,1,0,0,1
766,4,1024,0,4,1,0,1,0,0,0,...,1,1,0,0,1,0,1,0,0,1


In [18]:
y = df["price"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=42)

In [25]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)


    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAPE", mape)

    signature = infer_signature(X_test, predictions)

    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else: 
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    print(f"Model {model_name} logged with MSE: {mse}, MAE: {mae}, R2: {r2}, MAPE: {mape}")

### Experimento com Ridge Regression

In [21]:
with mlflow.start_run(run_name="Ridge Regression"):
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0]
    }

    ridge = Ridge()
    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", best_model.alpha)
    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_test, y_test)

2025/06/11 20:33:27 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


🏃 View run hilarious-owl-547 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/3040a384a83b433688f68f7b4a194c11
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run classy-duck-304 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/ef78a21556044e4fb28ddd2c238f03ea
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run thoughtful-skunk-468 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/83e41b0305fd409d97950f85f957ccfd
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run placid-conch-85 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/db50df061bbc4f47a9251f6f07d7c910
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


2025/06/11 20:33:47 INFO mlflow.models.model: Found the following environment variables used during model inference: [OPENAI_API_KEY_PERSONAL]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


Model ridge_regression logged with MSE: 3316545.045256343, MAE: 1167.1490161049967, R2: 0.62939127034036, MAPE: 0.21773072276221375
🏃 View run Ridge Regression at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/64acc428c39542f7a301d48a0f1d8c3b
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


🏃 View run delicate-cod-554 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/4ffe3fe099de466dadac39540c9bc9ba
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run thundering-croc-950 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/cce8642273e744aebd8ed967001572ee
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run fearless-lamb-990 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/a2d4b070d68946e8b1a989623645da93
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run charming-rook-328 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/23125b14bf054b5ab36de8f518e631fe
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run grandiose-auk-278 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/3c09

### Decision tree regression

In [23]:
with mlflow.start_run(run_name="Decision Tree Regression"):
    param_grid = {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
    }

    tree = DecisionTreeRegressor(random_state=42)    
    grid_search = GridSearchCV(tree, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_min_samples_split", best_model.min_samples_split)

    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_test, y_test)

2025/06/11 20:40:38 INFO mlflow.sklearn.utils: Logging the 5 best runs, 7 runs will be omitted.


🏃 View run clumsy-fowl-695 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/cf835546a32c47c3a4daeccad5e6e050
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


Model ridge_regression logged with MSE: 4209259.010082455, MAE: 1166.5048875727448, R2: 0.5296345705401153, MAPE: 0.18397941173326327
🏃 View run Decision Tree Regression at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/9b894e108d924400a7958606e5b6a7c7
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


🏃 View run peaceful-cod-173 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/ab69480b7ace45d185d2523d7c5afac9
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0
🏃 View run stylish-zebra-961 at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/32f59724eeae4722b3d4689bc89be498
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


### MLP Regression

In [26]:
from sklearn.neural_network import MLPRegressor

with mlflow.start_run(run_name="MLP_Regressor"):
    param_grid = {
        'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50)],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001, 0.001, 0.01]
    }
    mlp = MLPRegressor(max_iter=500, random_state=42)
    grid_search = GridSearchCV(mlp, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_hidden_layer_sizes", best_model.hidden_layer_sizes)
    mlflow.log_param("best_activation", best_model.activation)
    mlflow.log_param("best_alpha", best_model.alpha)
    evaluate_and_log_model("sklearn", "MLP Regressor", best_model, X_test, y_test)

Model MLP Regressor logged with MSE: 3826638.8060325338, MAE: 1248.317311606274, R2: 0.5723906271683448, MAPE: 0.2263749135803876
🏃 View run MLP_Regressor at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/b5b14ffe3dc645f185de1fbbdbc4d388
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


### XGboost Regressor

In [27]:
with mlflow.start_run(run_name="XGBoost_Regressor"):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.1, 0.2, 0.3]
    }
    xgb = XGBRegressor(random_state=42, verbosity=0)
    grid_search = GridSearchCV(xgb, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    evaluate_and_log_model("xgboost", "XGBoost Regressor", best_model, X_test, y_test)

2025/06/11 20:49:03 INFO mlflow.sklearn.utils: Logging the 5 best runs, 43 runs will be omitted.
e:\BKP_COMPUTADOR\Documentos\POS - IA\MLOPS\Aula2\fiap-mlops\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [20:49:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


Model XGBoost Regressor logged with MSE: 3238380.5, MAE: 979.5505981445312, R2: 0.6381257772445679, MAPE: 0.15570877492427826
🏃 View run XGBoost_Regressor at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0/runs/80c3d8fb30de4facb15ea5af91c48896
🧪 View experiment at: https://dagshub.com/fabioebner/fiap-mlops.mlflow/#/experiments/0


## Model Registry

In [29]:
run_id = "80c3d8fb30de4facb15ea5af91c48896" #Id do melhor moelo

mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name="laptop-pricing-model-brl"
)

Successfully registered model 'laptop-pricing-model-brl'.
2025/06/11 20:54:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: laptop-pricing-model-brl, version 1
Created version '1' of model 'laptop-pricing-model-brl'.


<ModelVersion: aliases=[], creation_timestamp=1749686038626, current_stage='None', description='', last_updated_timestamp=1749686038626, name='laptop-pricing-model-brl', run_id='80c3d8fb30de4facb15ea5af91c48896', run_link='', source='mlflow-artifacts:/aab1e94b599042aa8f6be9bf3a5d4861/80c3d8fb30de4facb15ea5af91c48896/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>